
# Multi‑Frequency Comparison (L1/L2/L5)

This notebook compares reflector‑height estimates across **L1, L2, L5**.
We assume you have detrended, normalized signals per band vs \(x=\sin E\).
If your cleaned file only has one band, we’ll generate a **synthetic** multi‑band example to illustrate the workflow.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

# Wavelengths (m)
c = 299792458.0
f_L1 = 1575.42e6
f_L2 = 1227.60e6
f_L5 = 1176.45e6
lambda_L1 = c / f_L1
lambda_L2 = c / f_L2
lambda_L5 = c / f_L5

def height_from_freq(f_peak, band='L1'):
    wl = {'L1': lambda_L1, 'L2': lambda_L2, 'L5': lambda_L5}.get(band, lambda_L1)
    return 0.5 * wl * f_peak

def lomb_scargle_classic(x, y, freqs):
    y = y - np.nanmean(y)
    m = np.isfinite(x) & np.isfinite(y)
    x = x[m]; y = y[m]
    if x.size < 8:
        raise ValueError("Not enough points for Lomb–Scargle")
    w = 2.0 * np.pi * freqs[:, None]
    two_omega_x = 2.0 * w * x[None, :]
    tan2wtau = np.sum(np.sin(two_omega_x), axis=1) / np.sum(np.cos(two_omega_x), axis=1)
    tau = 0.5 * np.arctan(tan2wtau) / (2.0 * np.pi)
    wt = 2.0 * np.pi * (freqs[:, None]) * (x[None, :] - tau[:, None])
    cos_wt = np.cos(wt); sin_wt = np.sin(wt)
    C = np.sum(y[None, :] * cos_wt, axis=1); S = np.sum(y[None, :] * sin_wt, axis=1)
    CC = np.sum(cos_wt**2, axis=1); SS = np.sum(sin_wt**2, axis=1)
    P = 0.5 * (C**2/CC + S**2/SS)
    var_y = np.nanvar(y)
    return P/var_y if var_y>0 else P



## Load Multi‑Band Cleaned Data (or Synthesize)

Expected columns (if available):  
`sinE, snr_detrended_z_L1, snr_detrended_z_L2, snr_detrended_z_L5`

If not found, we create synthetic multi‑band signals sharing the **same geometry** (same \(\sin E\)) but different noise.


In [ ]:

def load_or_synthesize():
    path = Path('/mnt/data/cleaned_arcs_multi.csv')
    if path.exists():
        df = pd.read_csv(path)
        print('Loaded:', path)
        return df
    else:
        print('No multi-band file found; generating synthetic example.')
        rng = np.random.default_rng(1)
        sinE = np.sort(np.sin(np.deg2rad(np.linspace(5, 30, 500))))
        # Use the same f_true but different noise realizations; in practice, each band has its own residuals
        f_true = 20.0
        phi = 0.25
        y1 = np.cos(2*np.pi*f_true*sinE + phi) + 0.20*rng.standard_normal(len(sinE))
        y2 = np.cos(2*np.pi*f_true*sinE + phi) + 0.25*rng.standard_normal(len(sinE))
        y5 = np.cos(2*np.pi*f_true*sinE + phi) + 0.18*rng.standard_normal(len(sinE))
        return pd.DataFrame({
            'sinE': sinE,
            'snr_detrended_z_L1': y1,
            'snr_detrended_z_L2': y2,
            'snr_detrended_z_L5': y5,
        })

df = load_or_synthesize()
df.head()



## Lomb–Scargle per Band and Height Estimates


In [ ]:

fmin, fmax, M = 0.0, 60.0, 4000
freqs = np.linspace(fmin, fmax, M)

results = []
bands = []
if 'snr_detrended_z_L1' in df.columns:
    bands.append(('L1','snr_detrended_z_L1', lambda_L1))
if 'snr_detrended_z_L2' in df.columns:
    bands.append(('L2','snr_detrended_z_L2', lambda_L2))
if 'snr_detrended_z_L5' in df.columns:
    bands.append(('L5','snr_detrended_z_L5', lambda_L5))

for band, col, wl in bands:
    power = lomb_scargle_classic(df['sinE'].values, df[col].values, freqs)
    k0 = 5
    k_peak = k0 + int(np.argmax(power[k0:]))
    f_peak = freqs[k_peak]
    h_est = 0.5 * wl * f_peak
    results.append({'band': band, 'f_peak': f_peak, 'h_est_m': h_est})

res_df = pd.DataFrame(results)
res_df



## Plot Spectra and Compare Heights


In [ ]:

plt.figure(figsize=(8,5))
for band, col, wl in bands:
    power = lomb_scargle_classic(df['sinE'].values, df[col].values, freqs)
    plt.plot(freqs, power, label=f'{band}')

for _, row in res_df.iterrows():
    plt.axvline(row['f_peak'], ls='--', alpha=0.6)
plt.xlabel('Frequency [cycles per unit sin(E)]')
plt.ylabel('Normalized Power')
plt.title('Lomb–Scargle Spectra by Band')
plt.grid(True); plt.legend()
plt.show()

print('Height estimates by band (m):')
print(res_df)



## Discussion

- If the same **geometric reflection** dominates each band, the **peak frequency** \(f\\) should be similar across L1/L2/L5.  
  Height estimates differ only via the wavelength factor \(h = (\\lambda/2)f\).
- In real data, bands differ in **coherence**, **multipath contamination**, and **receiver SNR** — which can shift/blur peaks.
- A practical approach is to **cross‑validate**: require consistent peaks across bands and report a robust central estimate with spread.



## Next Steps
- Combine per‑arc, per‑band height estimates into **time series**.
- Add quality metrics (peak width, SNR thresholds) and outlier screening.
- Move toward application notebooks (snow depth, soil moisture, water level).
